In [1]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings("ignore")

from datetime import datetime

today = datetime.now().strftime("%Y-%m-%d")
games = pd.read_csv("predict/games_2024_2025.csv")
teams = pd.read_csv("predict/kenpom_2024_2025.csv").dropna(subset="Team")
teams = teams[teams['Rk'] != "Rk"]

In [2]:
# Define the regex pattern
pattern = r"(\D+?)\s(\d+),\s(?:\d+\s)?(\D+?)\s(\d+)"

# Function to extract teams and scores using regex
def extract_teams_scores(text):
    match = re.search(pattern, text)
    if match:
        return pd.Series({
            'Home Team': match.group(1).strip(),
            'Home Score': int(match.group(2)),
            'Away Team': match.group(3).strip(),
            'Away Score': int(match.group(4))
        })
    else:
        return pd.Series({'Home Team': None, 'Home Score': None, 'Away Team': None, 'Away Score': None})

# Apply the function to each row in the 'GameInfo' column
games[['Home Team', 'Home Score', 'Away Team', 'Away Score']] = games['Game'].apply(extract_teams_scores)

# Display the DataFrame
games['home_score'] = games['Home Score']
games['home_team'] = games['Home Team']
games['away_team'] = games['Away Team']
games['away_score'] = games['Away Score']
games['total_score'] = games['away_score'] + games['home_score']

games = games[["date", "home_score", "home_team", "away_score", "away_team", "total_score"]]

In [3]:
games

,date,home_score,home_team,away_score,away_team,total_score
0,11/4/24,101.0,Gonzaga,63.0,Baylor,164.0
1,11/4/24,80.0,Ohio St.,72.0,Texas,152.0
2,11/4/24,64.0,UCF,61.0,Texas A&M,125.0
3,11/4/24,83.0,Memphis,75.0,Missouri,158.0
4,11/4/24,85.0,Santa Clara,78.0,Saint Louis,163.0
...,...,...,...,...,...,...
3675,1/21/25,81.0,UAB,78.0,UTSA,159.0
3676,1/21/25,79.0,Northern Iowa,68.0,Missouri St.,147.0
3677,1/21/25,90.0,Akron,58.0,Buffalo,148.0
3678,1/21/25,84.0,Miami OH,76.0,Bowling Green,160.0


In [4]:
teams['date'] = pd.to_datetime(teams['date'])

# Sort by date
teams = teams.sort_values(by=['Team', 'date']).reset_index(drop=True)

# Define columns for rolling calculations
numeric_columns = ['Rk', 'AdjEM', 'AdjO', 'AdjO_Rk', 'AdjD', 'AdjD_Rk', 'AdjT', 'AdjT_Rk']

# Define rolling windows
rolling_windows = [1, 3, 5, 10, 20]

# Create rolling features
for window in rolling_windows:
    for col in numeric_columns:
        teams[col] = pd.to_numeric(teams[col])
        # Create rolling mean feature
        teams[f'{col}_rolling_{window}'] = (
            teams.groupby('Team')[col]  # Group by 'Team'
            .transform(lambda x: x.rolling(window, min_periods=1).mean())
        )

today_teams = teams[teams['date'] == today]

In [5]:
from datetime import timedelta
# Ensure date columns are in datetime format
games['date'] = pd.to_datetime(games['date'])
teams['date'] = pd.to_datetime(teams['date'])
# Function to merge games with rankings using date and team names with date tolerance
def merge_with_tolerance(games_df, rankings_df, days_tolerance=2):
    # Merge exact date matches
    merged_df = games_df.merge(rankings_df, how='left', left_on=['date', 'home_team'], right_on=['date', 'Team'])
    merged_df = merged_df.merge(rankings_df, how='left', left_on=['date', 'away_team'], right_on=['date', 'Team'], suffixes=('_home', '_away'))

    # Try matching dates up to `days_tolerance` days before if no exact match is found
    for days_back in range(1, days_tolerance + 1):
        unmatched_home = merged_df[merged_df['Rk_home'].isna()]
        unmatched_away = merged_df[merged_df['Rk_away'].isna()]

        # Home team fallback match
        fallback_home = games_df[games_df['date'].isin(unmatched_home['date'] - timedelta(days=days_back))]
        fallback_merged_home = fallback_home.merge(rankings_df, how='left', left_on=['date', 'home_team'], right_on=['date', 'Team'])
        
        # Away team fallback match
        fallback_away = games_df[games_df['date'].isin(unmatched_away['date'] - timedelta(days=days_back))]
        fallback_merged_away = fallback_away.merge(rankings_df, how='left', left_on=['date', 'away_team'], right_on=['date', 'Team'])

        # Update original DataFrame with fallback data
        merged_df.update(fallback_merged_home)
        merged_df.update(fallback_merged_away)
    
    return merged_df

# Apply the function
merged_df = merge_with_tolerance(games, teams)

# Display the merged DataFrame
print(merged_df)

           date  home_score         home_team  away_score          away_team  \
0    2024-11-04       101.0           Gonzaga        63.0             Baylor   
1    2024-11-04        80.0          Ohio St.        72.0              Texas   
2    2024-11-04        64.0               UCF        61.0          Texas A&M   
3    2024-11-04        83.0           Memphis        75.0           Missouri   
4    2024-11-04        85.0       Santa Clara        78.0        Saint Louis   
...         ...         ...               ...         ...                ...   
3675 2025-01-21        81.0               UAB        78.0               UTSA   
3676 2025-01-21        79.0     Northern Iowa        68.0       Missouri St.   
3677 2025-01-21        90.0             Akron        58.0            Buffalo   
3678 2025-01-21        84.0          Miami OH        76.0      Bowling Green   
3679 2025-01-21        72.0  Western Michigan        70.0  Northern Illinois   

      total_score  Rk_home         Team

In [6]:
data = merged_df
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by="date", ascending=True)

data["home_opp_score_rank"] = data["Rk_away"] - data['home_score']
data['away_opp_score_rank'] = data['Rk_home'] - data['away_score']
data['home_sos'] = data['Rk_away']
data['away_sos'] = data['Rk_home']


home_df = data[['date', 'home_team', 'home_score', 'away_score', 'home_sos', "home_opp_score_rank"]].rename(columns={
    'home_team': 'team', 'home_score': 'score', 'away_score': 'opponent_score', 'home_sos': 'sos', "home_opp_score_rank": "opp_score_rank"
})
away_df = data[['date', 'away_team', 'away_score', 'home_score','away_sos', "away_opp_score_rank"]].rename(columns={
    'away_team': 'team', 'away_score': 'score', 'home_score': 'opponent_score','away_sos': 'sos', "away_opp_score_rank": "opp_score_rank"
})
games = pd.concat([home_df, away_df])
games.sort_values(by=['team', 'date'], inplace=True)
games = games[~games['team'].isna()].drop_duplicates(subset=["date", "team"])

In [7]:
# Define the rolling windows
rolling_windows = [1, 3, 7]

# Calculate rolling averages for each window and store as separate columns
for window in rolling_windows:
    games[f'rolling_avg_score_{window}'] = games.groupby('team')['score'].shift().rolling(window=window, min_periods=1).mean()
    games[f'rolling_avg_score_allowed_{window}'] = games.groupby('team')['opponent_score'].shift().rolling(window=window, min_periods=1).mean()
    games[f'rolling_sos_{window}'] = games.groupby('team')['sos'].shift().rolling(window=window, min_periods=1).mean()
    games[f'rolling_opp_score_rank_{window}'] = games.groupby('team')['opp_score_rank'].shift().rolling(window=window, min_periods=1).mean()


games['days_since_last_game'] = games.groupby('team')['date'].diff().dt.days

# Keep only necessary columns
columns_to_keep = ['date', 'team', 'days_since_last_game', 'sos'] + \
                  [f'rolling_avg_score_{w}' for w in rolling_windows] + \
                  [f'rolling_sos_{w}' for w in rolling_windows] + \
                  [f'rolling_avg_score_allowed_{w}' for w in rolling_windows] + \
                  [f'rolling_opp_score_rank_{w}' for w in rolling_windows]
games = games[columns_to_keep]

In [8]:
games = games.sort_values("date", ascending=True).drop_duplicates(subset="team", keep="last")

In [9]:
daily_games = pd.read_csv("predict/daily-games.csv")

In [10]:
# First, merge `daily_games` with `today_teams` based on home and away teams separately
pred_df = pd.merge(daily_games, today_teams, left_on="home", right_on="Team", suffixes=("", "_home"))

# Merge `today_teams` data for the away team
pred_df = pd.merge(pred_df, today_teams, left_on="away", right_on="Team", suffixes=("", "_away"))

# # Rename columns in `games` with `home_` prefix, then merge for the home team data
pred_df_2 = pd.merge(daily_games, games, left_on="home", right_on="team", how="left", suffixes=("", "_home"))

# Rename columns in `games` with `away_` prefix, then merge for the away team data
pred_df_2 = pd.merge(pred_df_2, games, left_on="away", right_on="team", how="left", suffixes=("", "_away"))

In [11]:
preds = pd.merge(pred_df, pred_df_2, on=["home","away"])

In [12]:
home_cols = ['Rk', 'Conf', 'AdjEM', 'AdjO', 'AdjO_Rk', 'AdjD', 'AdjD_Rk', 'AdjT', 'AdjT_Rk', 'Rk_rolling_1',
       'AdjEM_rolling_1', 'AdjO_rolling_1', 'AdjO_Rk_rolling_1',
       'AdjD_rolling_1', 'AdjD_Rk_rolling_1', 'AdjT_rolling_1',
       'AdjT_Rk_rolling_1', 'Rk_rolling_3', 'AdjEM_rolling_3',
       'AdjO_rolling_3', 'AdjO_Rk_rolling_3', 'AdjD_rolling_3',
       'AdjD_Rk_rolling_3', 'AdjT_rolling_3', 'AdjT_Rk_rolling_3',
       'Rk_rolling_5', 'AdjEM_rolling_5', 'AdjO_rolling_5',
       'AdjO_Rk_rolling_5', 'AdjD_rolling_5', 'AdjD_Rk_rolling_5',
       'AdjT_rolling_5', 'AdjT_Rk_rolling_5', 'Rk_rolling_10',
       'AdjEM_rolling_10', 'AdjO_rolling_10', 'AdjO_Rk_rolling_10',
       'AdjD_rolling_10', 'AdjD_Rk_rolling_10', 'AdjT_rolling_10',
       'AdjT_Rk_rolling_10', 'Rk_rolling_20', 'AdjEM_rolling_20',
       'AdjO_rolling_20', 'AdjO_Rk_rolling_20', 'AdjD_rolling_20',
       'AdjD_Rk_rolling_20', 'AdjT_rolling_20', 'AdjT_Rk_rolling_20']
home_cols_roll = ['days_since_last_game', 'rolling_avg_score_1',
       'rolling_avg_score_3', 'rolling_avg_score_7', 'rolling_sos_1',
       'rolling_sos_3', 'rolling_sos_7', 'rolling_avg_score_allowed_1',
       'rolling_avg_score_allowed_3', 'rolling_avg_score_allowed_7',
       'rolling_opp_score_rank_1', 'rolling_opp_score_rank_3',
       'rolling_opp_score_rank_7', 'sos']
for col in home_cols:
    preds[f'{col}_home'] = preds[col]
for col in home_cols_roll:
    preds[f'home_{col}'] = preds[col]

away_cols_roll = ['days_since_last_game_away', 'rolling_avg_score_1_away',
       'rolling_avg_score_3_away', 'rolling_avg_score_7_away',
       'rolling_sos_1_away', 'rolling_sos_3_away', 'rolling_sos_7_away',
       'rolling_avg_score_allowed_1_away',
       'rolling_avg_score_allowed_3_away',
       'rolling_avg_score_allowed_7_away',
       'rolling_opp_score_rank_1_away', 'rolling_opp_score_rank_3_away',
       'rolling_opp_score_rank_7_away','sos_away']
for col in away_cols_roll:
    preds[f"away_{col.strip('_away')}"] = preds[col]

In [13]:
preds = preds[['home', 'away', 'Rk_home','Conf_home', 'AdjEM_home', 'AdjO_home',
       'AdjO_Rk_home', 'AdjD_home', 'AdjD_Rk_home', 'AdjT_home',
       'AdjT_Rk_home', 'Rk_rolling_1_home',
       'AdjEM_rolling_1_home', 'AdjO_rolling_1_home',
       'AdjO_Rk_rolling_1_home', 'AdjD_rolling_1_home',
       'AdjD_Rk_rolling_1_home', 'AdjT_rolling_1_home',
       'AdjT_Rk_rolling_1_home', 'Rk_rolling_3_home',
       'AdjEM_rolling_3_home', 'AdjO_rolling_3_home',
       'AdjO_Rk_rolling_3_home', 'AdjD_rolling_3_home',
       'AdjD_Rk_rolling_3_home', 'AdjT_rolling_3_home',
       'AdjT_Rk_rolling_3_home', 'Rk_rolling_5_home',
       'AdjEM_rolling_5_home', 'AdjO_rolling_5_home',
       'AdjO_Rk_rolling_5_home', 'AdjD_rolling_5_home',
       'AdjD_Rk_rolling_5_home', 'AdjT_rolling_5_home',
       'AdjT_Rk_rolling_5_home', 'Rk_rolling_10_home',
       'AdjEM_rolling_10_home', 'AdjO_rolling_10_home',
       'AdjO_Rk_rolling_10_home', 'AdjD_rolling_10_home',
       'AdjD_Rk_rolling_10_home', 'AdjT_rolling_10_home',
       'AdjT_Rk_rolling_10_home', 'Rk_rolling_20_home',
       'AdjEM_rolling_20_home', 'AdjO_rolling_20_home',
       'AdjO_Rk_rolling_20_home', 'AdjD_rolling_20_home',
       'AdjD_Rk_rolling_20_home', 'AdjT_rolling_20_home',
       'AdjT_Rk_rolling_20_home', 'Rk_away', 'Conf_away',
       'AdjEM_away', 'AdjO_away', 'AdjO_Rk_away', 'AdjD_away',
       'AdjD_Rk_away', 'AdjT_away', 'AdjT_Rk_away',
       'Rk_rolling_1_away', 'AdjEM_rolling_1_away', 'AdjO_rolling_1_away',
       'AdjO_Rk_rolling_1_away', 'AdjD_rolling_1_away',
       'AdjD_Rk_rolling_1_away', 'AdjT_rolling_1_away',
       'AdjT_Rk_rolling_1_away', 'Rk_rolling_3_away',
       'AdjEM_rolling_3_away', 'AdjO_rolling_3_away',
       'AdjO_Rk_rolling_3_away', 'AdjD_rolling_3_away',
       'AdjD_Rk_rolling_3_away', 'AdjT_rolling_3_away',
       'AdjT_Rk_rolling_3_away', 'Rk_rolling_5_away',
       'AdjEM_rolling_5_away', 'AdjO_rolling_5_away',
       'AdjO_Rk_rolling_5_away', 'AdjD_rolling_5_away',
       'AdjD_Rk_rolling_5_away', 'AdjT_rolling_5_away',
       'AdjT_Rk_rolling_5_away', 'Rk_rolling_10_away',
       'AdjEM_rolling_10_away', 'AdjO_rolling_10_away',
       'AdjO_Rk_rolling_10_away', 'AdjD_rolling_10_away',
       'AdjD_Rk_rolling_10_away', 'AdjT_rolling_10_away',
       'AdjT_Rk_rolling_10_away', 'Rk_rolling_20_away',
       'AdjEM_rolling_20_away', 'AdjO_rolling_20_away',
       'AdjO_Rk_rolling_20_away', 'AdjD_rolling_20_away',
       'AdjD_Rk_rolling_20_away', 'AdjT_rolling_20_away',
       'AdjT_Rk_rolling_20_away',  'home_sos',
       'away_sos', 'home_days_since_last_game',
       'home_rolling_avg_score_1', 'home_rolling_avg_score_3',
       'home_rolling_avg_score_7', 'home_rolling_sos_1',
       'home_rolling_sos_3', 'home_rolling_sos_7',
       'home_rolling_avg_score_allowed_1',
       'home_rolling_avg_score_allowed_3',
       'home_rolling_avg_score_allowed_7',
       'home_rolling_opp_score_rank_1', 'home_rolling_opp_score_rank_3',
       'home_rolling_opp_score_rank_7', 'away_days_since_last_game',
       'away_rolling_avg_score_1', 'away_rolling_avg_score_3',
       'away_rolling_avg_score_7', 'away_rolling_sos_1',
       'away_rolling_sos_3', 'away_rolling_sos_7',
       'away_rolling_avg_score_allowed_1',
       'away_rolling_avg_score_allowed_3',
       'away_rolling_avg_score_allowed_7',
       'away_rolling_opp_score_rank_1', 'away_rolling_opp_score_rank_3',
       'away_rolling_opp_score_rank_7']]

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
import pandas as pd
import joblib
import os

categorical_columns = ['Conf_home', 'Conf_away']
target_columns = ['home_score', 'away_score', 'total', 'margin']
model_dir = "lgb_models"
high_outlier_thresholds = {'home_score': 91, 'away_score': 78, 'total': 167}
low_outlier_thresholds = {'home_score': 56, 'away_score': 52, 'total': 118}
for cat_col in categorical_columns:
    le = LabelEncoder()
    preds[cat_col] = le.fit_transform(preds[cat_col])

features_for_outlier_detection = preds.drop(columns=["home","away"]).fillna(preds.mean())

# Apply Isolation Forest for outlier detection and add outlier score as a feature
iso_forest = IsolationForest(contamination=0.1, random_state=42)
iso_forest.fit(features_for_outlier_detection)
preds['outlier_score'] = iso_forest.decision_function(features_for_outlier_detection)

for target_column in target_columns:
    model_path = os.path.join(model_dir, f"{target_column}_best_model.joblib")
    model = joblib.load(model_path)


    preds[f"{target_column}_prediction"] = model.predict(preds[['Rk_home','Conf_home', 'AdjEM_home', 'AdjO_home',
       'AdjO_Rk_home', 'AdjD_home', 'AdjD_Rk_home', 'AdjT_home',
       'AdjT_Rk_home', 'Rk_rolling_1_home',
       'AdjEM_rolling_1_home', 'AdjO_rolling_1_home',
       'AdjO_Rk_rolling_1_home', 'AdjD_rolling_1_home',
       'AdjD_Rk_rolling_1_home', 'AdjT_rolling_1_home',
       'AdjT_Rk_rolling_1_home', 'Rk_rolling_3_home',
       'AdjEM_rolling_3_home', 'AdjO_rolling_3_home',
       'AdjO_Rk_rolling_3_home', 'AdjD_rolling_3_home',
       'AdjD_Rk_rolling_3_home', 'AdjT_rolling_3_home',
       'AdjT_Rk_rolling_3_home', 'Rk_rolling_5_home',
       'AdjEM_rolling_5_home', 'AdjO_rolling_5_home',
       'AdjO_Rk_rolling_5_home', 'AdjD_rolling_5_home',
       'AdjD_Rk_rolling_5_home', 'AdjT_rolling_5_home',
       'AdjT_Rk_rolling_5_home', 'Rk_rolling_10_home',
       'AdjEM_rolling_10_home', 'AdjO_rolling_10_home',
       'AdjO_Rk_rolling_10_home', 'AdjD_rolling_10_home',
       'AdjD_Rk_rolling_10_home', 'AdjT_rolling_10_home',
       'AdjT_Rk_rolling_10_home', 'Rk_rolling_20_home',
       'AdjEM_rolling_20_home', 'AdjO_rolling_20_home',
       'AdjO_Rk_rolling_20_home', 'AdjD_rolling_20_home',
       'AdjD_Rk_rolling_20_home', 'AdjT_rolling_20_home',
       'AdjT_Rk_rolling_20_home', 'Rk_away', 'Conf_away',
       'AdjEM_away', 'AdjO_away', 'AdjO_Rk_away', 'AdjD_away',
       'AdjD_Rk_away', 'AdjT_away', 'AdjT_Rk_away',
       'Rk_rolling_1_away', 'AdjEM_rolling_1_away', 'AdjO_rolling_1_away',
       'AdjO_Rk_rolling_1_away', 'AdjD_rolling_1_away',
       'AdjD_Rk_rolling_1_away', 'AdjT_rolling_1_away',
       'AdjT_Rk_rolling_1_away', 'Rk_rolling_3_away',
       'AdjEM_rolling_3_away', 'AdjO_rolling_3_away',
       'AdjO_Rk_rolling_3_away', 'AdjD_rolling_3_away',
       'AdjD_Rk_rolling_3_away', 'AdjT_rolling_3_away',
       'AdjT_Rk_rolling_3_away', 'Rk_rolling_5_away',
       'AdjEM_rolling_5_away', 'AdjO_rolling_5_away',
       'AdjO_Rk_rolling_5_away', 'AdjD_rolling_5_away',
       'AdjD_Rk_rolling_5_away', 'AdjT_rolling_5_away',
       'AdjT_Rk_rolling_5_away', 'Rk_rolling_10_away',
       'AdjEM_rolling_10_away', 'AdjO_rolling_10_away',
       'AdjO_Rk_rolling_10_away', 'AdjD_rolling_10_away',
       'AdjD_Rk_rolling_10_away', 'AdjT_rolling_10_away',
       'AdjT_Rk_rolling_10_away', 'Rk_rolling_20_away',
       'AdjEM_rolling_20_away', 'AdjO_rolling_20_away',
       'AdjO_Rk_rolling_20_away', 'AdjD_rolling_20_away',
       'AdjD_Rk_rolling_20_away', 'AdjT_rolling_20_away',
       'AdjT_Rk_rolling_20_away',  'home_sos',
       'away_sos', 'home_days_since_last_game',
       'home_rolling_avg_score_1', 'home_rolling_avg_score_3',
       'home_rolling_avg_score_7', 'home_rolling_sos_1',
       'home_rolling_sos_3', 'home_rolling_sos_7',
       'home_rolling_avg_score_allowed_1',
       'home_rolling_avg_score_allowed_3',
       'home_rolling_avg_score_allowed_7',
       'home_rolling_opp_score_rank_1', 'home_rolling_opp_score_rank_3',
       'home_rolling_opp_score_rank_7', 'away_days_since_last_game',
       'away_rolling_avg_score_1', 'away_rolling_avg_score_3',
       'away_rolling_avg_score_7', 'away_rolling_sos_1',
       'away_rolling_sos_3', 'away_rolling_sos_7',
       'away_rolling_avg_score_allowed_1',
       'away_rolling_avg_score_allowed_3',
       'away_rolling_avg_score_allowed_7',
       'away_rolling_opp_score_rank_1', 'away_rolling_opp_score_rank_3',
       'away_rolling_opp_score_rank_7', 'outlier_score']])
model_dir = "xgb_models"
home_win_model_path = os.path.join(model_dir, "home_win_best_model.joblib")
away_win_model_path = os.path.join(model_dir, "away_win_best_model.joblib")
home_win_model = joblib.load(home_win_model_path)
away_win_model = joblib.load(away_win_model_path)

preds["home_prob_prediction"] = home_win_model.predict_proba(preds[['Rk_home','Conf_home', 'AdjEM_home', 'AdjO_home',
       'AdjO_Rk_home', 'AdjD_home', 'AdjD_Rk_home', 'AdjT_home',
       'AdjT_Rk_home', 'Rk_rolling_1_home',
       'AdjEM_rolling_1_home', 'AdjO_rolling_1_home',
       'AdjO_Rk_rolling_1_home', 'AdjD_rolling_1_home',
       'AdjD_Rk_rolling_1_home', 'AdjT_rolling_1_home',
       'AdjT_Rk_rolling_1_home', 'Rk_rolling_3_home',
       'AdjEM_rolling_3_home', 'AdjO_rolling_3_home',
       'AdjO_Rk_rolling_3_home', 'AdjD_rolling_3_home',
       'AdjD_Rk_rolling_3_home', 'AdjT_rolling_3_home',
       'AdjT_Rk_rolling_3_home', 'Rk_rolling_5_home',
       'AdjEM_rolling_5_home', 'AdjO_rolling_5_home',
       'AdjO_Rk_rolling_5_home', 'AdjD_rolling_5_home',
       'AdjD_Rk_rolling_5_home', 'AdjT_rolling_5_home',
       'AdjT_Rk_rolling_5_home', 'Rk_rolling_10_home',
       'AdjEM_rolling_10_home', 'AdjO_rolling_10_home',
       'AdjO_Rk_rolling_10_home', 'AdjD_rolling_10_home',
       'AdjD_Rk_rolling_10_home', 'AdjT_rolling_10_home',
       'AdjT_Rk_rolling_10_home', 'Rk_rolling_20_home',
       'AdjEM_rolling_20_home', 'AdjO_rolling_20_home',
       'AdjO_Rk_rolling_20_home', 'AdjD_rolling_20_home',
       'AdjD_Rk_rolling_20_home', 'AdjT_rolling_20_home',
       'AdjT_Rk_rolling_20_home', 'Rk_away', 'Conf_away',
       'AdjEM_away', 'AdjO_away', 'AdjO_Rk_away', 'AdjD_away',
       'AdjD_Rk_away', 'AdjT_away', 'AdjT_Rk_away',
       'Rk_rolling_1_away', 'AdjEM_rolling_1_away', 'AdjO_rolling_1_away',
       'AdjO_Rk_rolling_1_away', 'AdjD_rolling_1_away',
       'AdjD_Rk_rolling_1_away', 'AdjT_rolling_1_away',
       'AdjT_Rk_rolling_1_away', 'Rk_rolling_3_away',
       'AdjEM_rolling_3_away', 'AdjO_rolling_3_away',
       'AdjO_Rk_rolling_3_away', 'AdjD_rolling_3_away',
       'AdjD_Rk_rolling_3_away', 'AdjT_rolling_3_away',
       'AdjT_Rk_rolling_3_away', 'Rk_rolling_5_away',
       'AdjEM_rolling_5_away', 'AdjO_rolling_5_away',
       'AdjO_Rk_rolling_5_away', 'AdjD_rolling_5_away',
       'AdjD_Rk_rolling_5_away', 'AdjT_rolling_5_away',
       'AdjT_Rk_rolling_5_away', 'Rk_rolling_10_away',
       'AdjEM_rolling_10_away', 'AdjO_rolling_10_away',
       'AdjO_Rk_rolling_10_away', 'AdjD_rolling_10_away',
       'AdjD_Rk_rolling_10_away', 'AdjT_rolling_10_away',
       'AdjT_Rk_rolling_10_away', 'Rk_rolling_20_away',
       'AdjEM_rolling_20_away', 'AdjO_rolling_20_away',
       'AdjO_Rk_rolling_20_away', 'AdjD_rolling_20_away',
       'AdjD_Rk_rolling_20_away', 'AdjT_rolling_20_away',
       'AdjT_Rk_rolling_20_away',  'home_sos',
       'away_sos', 'home_days_since_last_game',
       'home_rolling_avg_score_1', 'home_rolling_avg_score_3',
       'home_rolling_avg_score_7', 'home_rolling_sos_1',
       'home_rolling_sos_3', 'home_rolling_sos_7',
       'home_rolling_avg_score_allowed_1',
       'home_rolling_avg_score_allowed_3',
       'home_rolling_avg_score_allowed_7',
       'home_rolling_opp_score_rank_1', 'home_rolling_opp_score_rank_3',
       'home_rolling_opp_score_rank_7', 'away_days_since_last_game',
       'away_rolling_avg_score_1', 'away_rolling_avg_score_3',
       'away_rolling_avg_score_7', 'away_rolling_sos_1',
       'away_rolling_sos_3', 'away_rolling_sos_7',
       'away_rolling_avg_score_allowed_1',
       'away_rolling_avg_score_allowed_3',
       'away_rolling_avg_score_allowed_7',
       'away_rolling_opp_score_rank_1', 'away_rolling_opp_score_rank_3',
       'away_rolling_opp_score_rank_7']])[:, 1]
    
preds["away_prob_prediction"] = away_win_model.predict_proba(preds[['Rk_home','Conf_home', 'AdjEM_home', 'AdjO_home',
       'AdjO_Rk_home', 'AdjD_home', 'AdjD_Rk_home', 'AdjT_home',
       'AdjT_Rk_home', 'Rk_rolling_1_home',
       'AdjEM_rolling_1_home', 'AdjO_rolling_1_home',
       'AdjO_Rk_rolling_1_home', 'AdjD_rolling_1_home',
       'AdjD_Rk_rolling_1_home', 'AdjT_rolling_1_home',
       'AdjT_Rk_rolling_1_home', 'Rk_rolling_3_home',
       'AdjEM_rolling_3_home', 'AdjO_rolling_3_home',
       'AdjO_Rk_rolling_3_home', 'AdjD_rolling_3_home',
       'AdjD_Rk_rolling_3_home', 'AdjT_rolling_3_home',
       'AdjT_Rk_rolling_3_home', 'Rk_rolling_5_home',
       'AdjEM_rolling_5_home', 'AdjO_rolling_5_home',
       'AdjO_Rk_rolling_5_home', 'AdjD_rolling_5_home',
       'AdjD_Rk_rolling_5_home', 'AdjT_rolling_5_home',
       'AdjT_Rk_rolling_5_home', 'Rk_rolling_10_home',
       'AdjEM_rolling_10_home', 'AdjO_rolling_10_home',
       'AdjO_Rk_rolling_10_home', 'AdjD_rolling_10_home',
       'AdjD_Rk_rolling_10_home', 'AdjT_rolling_10_home',
       'AdjT_Rk_rolling_10_home', 'Rk_rolling_20_home',
       'AdjEM_rolling_20_home', 'AdjO_rolling_20_home',
       'AdjO_Rk_rolling_20_home', 'AdjD_rolling_20_home',
       'AdjD_Rk_rolling_20_home', 'AdjT_rolling_20_home',
       'AdjT_Rk_rolling_20_home', 'Rk_away', 'Conf_away',
       'AdjEM_away', 'AdjO_away', 'AdjO_Rk_away', 'AdjD_away',
       'AdjD_Rk_away', 'AdjT_away', 'AdjT_Rk_away',
       'Rk_rolling_1_away', 'AdjEM_rolling_1_away', 'AdjO_rolling_1_away',
       'AdjO_Rk_rolling_1_away', 'AdjD_rolling_1_away',
       'AdjD_Rk_rolling_1_away', 'AdjT_rolling_1_away',
       'AdjT_Rk_rolling_1_away', 'Rk_rolling_3_away',
       'AdjEM_rolling_3_away', 'AdjO_rolling_3_away',
       'AdjO_Rk_rolling_3_away', 'AdjD_rolling_3_away',
       'AdjD_Rk_rolling_3_away', 'AdjT_rolling_3_away',
       'AdjT_Rk_rolling_3_away', 'Rk_rolling_5_away',
       'AdjEM_rolling_5_away', 'AdjO_rolling_5_away',
       'AdjO_Rk_rolling_5_away', 'AdjD_rolling_5_away',
       'AdjD_Rk_rolling_5_away', 'AdjT_rolling_5_away',
       'AdjT_Rk_rolling_5_away', 'Rk_rolling_10_away',
       'AdjEM_rolling_10_away', 'AdjO_rolling_10_away',
       'AdjO_Rk_rolling_10_away', 'AdjD_rolling_10_away',
       'AdjD_Rk_rolling_10_away', 'AdjT_rolling_10_away',
       'AdjT_Rk_rolling_10_away', 'Rk_rolling_20_away',
       'AdjEM_rolling_20_away', 'AdjO_rolling_20_away',
       'AdjO_Rk_rolling_20_away', 'AdjD_rolling_20_away',
       'AdjD_Rk_rolling_20_away', 'AdjT_rolling_20_away',
       'AdjT_Rk_rolling_20_away',  'home_sos',
       'away_sos', 'home_days_since_last_game',
       'home_rolling_avg_score_1', 'home_rolling_avg_score_3',
       'home_rolling_avg_score_7', 'home_rolling_sos_1',
       'home_rolling_sos_3', 'home_rolling_sos_7',
       'home_rolling_avg_score_allowed_1',
       'home_rolling_avg_score_allowed_3',
       'home_rolling_avg_score_allowed_7',
       'home_rolling_opp_score_rank_1', 'home_rolling_opp_score_rank_3',
       'home_rolling_opp_score_rank_7', 'away_days_since_last_game',
       'away_rolling_avg_score_1', 'away_rolling_avg_score_3',
       'away_rolling_avg_score_7', 'away_rolling_sos_1',
       'away_rolling_sos_3', 'away_rolling_sos_7',
       'away_rolling_avg_score_allowed_1',
       'away_rolling_avg_score_allowed_3',
       'away_rolling_avg_score_allowed_7',
       'away_rolling_opp_score_rank_1', 'away_rolling_opp_score_rank_3',
       'away_rolling_opp_score_rank_7']])[:, 1]

In [15]:
preds.drop_duplicates(subset="home").to_csv(f"/Users/nickdimmitt/Desktop/lumber/ncaab/predictions-{today}.csv")

In [16]:
# preds.to_csv(f"/Users/nickdimmitt/Desktop/lumber/ncaab/predictions-2024-12-04.csv")

In [17]:
preds.drop_duplicates()

,home,away,Rk_home,Conf_home,AdjEM_home,AdjO_home,AdjO_Rk_home,AdjD_home,AdjD_Rk_home,AdjT_home,...,away_rolling_opp_score_rank_1,away_rolling_opp_score_rank_3,away_rolling_opp_score_rank_7,outlier_score,home_score_prediction,away_score_prediction,total_prediction,margin_prediction,home_prob_prediction,away_prob_prediction
0,Holy Cross,Army,314,17,-12.90,104.5,214,117.4,351,65.2,...,190.0,199.666667,199.833333,0.064431,72.469192,71.621253,144.441487,-1.560801,0.400830,0.621905
1,Houston,Utah,3,5,33.68,121.4,13,87.7,1,61.5,...,265.0,77.333333,26.333333,-0.045504,73.492778,60.205802,130.419677,10.047832,0.892291,0.110826
2,VMI,Furman,318,19,-13.77,99.7,300,113.5,312,67.0,...,66.0,146.333333,143.400000,0.063243,64.638412,72.571799,137.982447,-5.523315,0.180101,0.823555
3,Youngstown St.,Robert Morris,212,12,-3.95,98.9,313,102.8,97,69.6,...,-17.0,74.333333,91.833333,0.045975,73.320170,74.847771,150.596482,-1.360236,0.460634,0.560426
4,American,Lehigh,236,17,-5.78,103.8,225,109.6,236,63.4,...,231.0,223.666667,179.666667,0.059540,67.896422,64.884610,131.548984,4.000670,0.637902,0.340630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Portland,San Diego,317,23,-13.28,100.7,281,114.0,321,68.5,...,23.0,-23.333333,7.333333,0.031864,71.968394,73.542398,144.624825,-1.329386,0.449743,0.516997
94,Portland St.,Weber St.,228,7,-5.23,101.9,260,107.1,189,69.1,...,156.0,166.333333,147.833333,0.081153,70.175868,68.357952,139.473468,1.747140,0.516756,0.479062
95,UC San Diego,UC Santa Barbara,67,9,12.16,113.3,62,101.2,76,66.0,...,116.0,147.666667,115.428571,0.051698,71.341833,66.460987,137.356087,4.620342,0.753252,0.239751
96,Cal Baptist,Utah Valley,174,22,-0.57,108.0,145,108.6,218,65.5,...,113.0,75.333333,112.000000,0.095074,68.787411,70.790917,140.838276,-2.592766,0.434699,0.570359
